In [ ]:
import itertools
import string
import random
from functools import reduce
import ipywidgets as widgets


random.seed(1287)
inference_with_more_steps = False


def count_compatibility(item, pair_compatibility):
    return sum(
        pair_compatibility[tuple(sorted(comb))]
        for comb in itertools.combinations(item, 2)
    )


def randomchain(p, q, n_atoms, cut=0, max_iter=500, mix_induction_abduction=True):
    global inference_with_more_steps

    letters = list(string.ascii_lowercase)
    atoms = set(letters[:n_atoms])
    pair_compatibility = {
        tuple(sorted(comb)): (random.random() * 2 - 1)
        for comb in itertools.combinations(atoms, 2)
    }
    thepowerset: list[list[str]] = reduce(
        lambda result, x: result + [subset + [x] for subset in result], atoms, [[]]
    )[1:]
    thepowerset = [
        item
        for item in thepowerset
        if count_compatibility(item, pair_compatibility) >= cut
    ]

    hypothesis = set(p)
    consequence = set(q)
    iterations = 0
    inferences = []
    inference = None
    while not inference and iterations < max_iter:
        inference = generate_inference(
            hypothesis,
            consequence,
            thepowerset,
            mix_induction_abduction=mix_induction_abduction,
        )
        if inference is not None:
            iterations = 0
            inference = [sorted(element) for element in inference]
            inferences.extend(inference)
            inference = str(inference)
            print(pretty(inference))
        else:
            iterations += 1
    if iterations == max_iter:
        print("Inference not found!")

    all_inducing = all(
        e[0] == "<" for e in inferences if len(set(e) & set([">", "<"])) == 1
    )
    all_abducing = all(
        e[0] == ">" for e in inferences if len(set(e) & set([">", "<"])) == 1
    )
    return inferences, all_inducing, all_abducing, inference_with_more_steps


def generate_inference(
    hypothesis: set,
    consequence: set,
    thepowerset: list[list[str]],
    max_iter: int = 250,
    mix_induction_abduction: bool = True,
    max_attempts: int = 200,
):
    global inference_with_more_steps

    attempts = 0
    inference_with_more_steps = False
    powerset: list[list[str]] = thepowerset[:]
    inference: list[set] = [hypothesis]
    inducing = random.choice([False, True])
    while True:
        if attempts >= max_attempts:
            return None

        rule_matched = False
        last_element = set(inference[-1])
        element = set()
        listelement = list()

        iterations = 0
        while not rule_matched and iterations < max_iter:
            listelement = random.choice(powerset)
            element = set(listelement)
            if inducing:
                rule_matched = last_element < element  # is proper subset
            else:
                rule_matched = last_element > element  # is proper superset
            iterations += 1

        if not rule_matched:
            # try abduction
            inducing = not inducing
            attempts += 1
            continue

        operator = "<" if inducing else ">"
        inference.extend([operator, element])
        powerset.remove(listelement)

        if mix_induction_abduction:
            inducing = not inducing

        if element == consequence:
            return inference
        else:
            can_go_on = False
            if inducing:
                can_go_on = any(element < set(myset) for myset in powerset)
            else:
                can_go_on = any(element > set(myset) for myset in powerset)
            inference_with_more_steps = can_go_on
            if not can_go_on:
                return None


def pretty(inference):
    replacements = {
        "[[": "[",
        "]]": "]",
        "[": "{",
        "]": "}",
        ",": "+",
        "'": "",
        " ": "",
        "+{<}+": " < ",
        "+{>}+": " > ",
    }
    for key, value in replacements.items():
        inference = inference.replace(key, value)
    return inference


widgets.interact(
    randomchain,
    p="ad",
    q="acde",
    n_atoms=widgets.IntSlider(min=3, max=26, step=1, value=5),
    cut=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0),
    max_iter=widgets.IntSlider(min=10, max=1000, step=1, value=10),
    mix_induction_abduction=widgets.Checkbox(value=False),
)

interactive(children=(Text(value='ad', description='p'), Text(value='acde', description='q'), IntSlider(value=…

<function __main__.randomchain(p, q, n_atoms, cut=0, max_iter=500, mix_induction_abduction=True)>